# Imports

In [2]:
import numpy as np
import pandas as pd

from prophet import Prophet
import time
from prophet.plot import plot_plotly, plot_components_plotly


In [3]:
dollar = pd.read_csv('DS_market_data/dollar_value_table.csv', index_col=0)
pd_sales = pd.read_csv('DS_market_data\item_sales.zip')
calendar= pd.read_csv('DS_market_data\daily_calendar_with_events.csv')
clusters=pd.read_csv('DS_market_data\model_results_Tribeca.csv', index_col=0)

# Data preparation

For a beta version of our model, we have decided to select only a some items to be predicted. We will select one store store (the one with most sales) and select the item that were sold the most in the last 30 days 

In [7]:
clusters.head()

,Unnamed: 0,item,frequency,recency,frequency_ln,recency_ln,Cluster,Cluster_name
0,0,SUPERMARKET_3_703,57,1,4.060443,0.693147,Cluster 4,New Items
1,1,ACCESORIES_2_015,127,22,4.852030,3.135494,Cluster 0,Emerging/Fading Items
2,2,SUPERMARKET_2_086,4767,1,8.469682,0.693147,Cluster 2,Hot Items
3,3,HOME_&_GARDEN_1_348,5752,1,8.657477,0.693147,Cluster 2,Hot Items
4,4,ACCESORIES_1_165,712,1,6.569481,0.693147,Cluster 4,New Items


In [5]:
# Adding a store column to the dollar df
clusters_dict = (dict(zip(clusters.item, clusters.Cluster_name)))
store_dict= dict(zip(pd_sales.id, pd_sales.store))
dollar["store"] = dollar["id"].map(store_dict)
dollar["Cluster_name"] = dollar["item"].map(clusters_dict)

In [6]:
dollar.head()

,id,unit_sold,price,revenue,item,date,store,Cluster_name
0,ACCESORIES_1_001_NYC_1,0,11.019346,0.0,ACCESORIES_1_001,2011-01-29,Greenwich_Village,New Items
1,ACCESORIES_1_002_NYC_1,0,5.280100,0.0,ACCESORIES_1_002,2011-01-29,Greenwich_Village,New Items
2,ACCESORIES_1_003_NYC_1,0,3.950100,0.0,ACCESORIES_1_003,2011-01-29,Greenwich_Village,New Items
3,ACCESORIES_1_004_NYC_1,0,6.022636,0.0,ACCESORIES_1_004,2011-01-29,Greenwich_Village,Hot Items
4,ACCESORIES_1_005_NYC_1,0,3.909906,0.0,ACCESORIES_1_005,2011-01-29,Greenwich_Village,Common Items


In [102]:
# From task 1 we know that the store with the most sales is tribeca, thus I will take the 30 last days of the df an look what items were the most sold in that time period
Tribeca = dollar[((dollar.store=='Tribeca') & (dollar.Cluster_name == 'Hot Items'))]
Tribeca.head()

,id,unit_sold,price,revenue,item,date,store,Cluster_name
6101,ACCESORIES_1_004_NYC_3,0,6.019134,0.000000,ACCESORIES_1_004,2011-01-29,Tribeca,Hot Items
6105,ACCESORIES_1_008_NYC_3,11,0.629132,6.920449,ACCESORIES_1_008,2011-01-29,Tribeca,Hot Items
6111,ACCESORIES_1_014_NYC_3,0,1.341551,0.000000,ACCESORIES_1_014,2011-01-29,Tribeca,Hot Items
6112,ACCESORIES_1_015_NYC_3,22,0.939712,20.673668,ACCESORIES_1_015,2011-01-29,Tribeca,Hot Items
6113,ACCESORIES_1_016_NYC_3,21,0.939620,19.732010,ACCESORIES_1_016,2011-01-29,Tribeca,Hot Items


In [103]:
Tribeca.item.nunique()

567

In [30]:
calendar.tail(30)

,date,weekday,weekday_int,d,event
1883,2016-03-26,Saturday,1,d_1884,NaN
1884,2016-03-27,Sunday,2,d_1885,Easter
1885,2016-03-28,Monday,3,d_1886,NaN
1886,2016-03-29,Tuesday,4,d_1887,NaN
1887,2016-03-30,Wednesday,5,d_1888,NaN
1888,2016-03-31,Thursday,6,d_1889,NaN
1889,2016-04-01,Friday,7,d_1890,NaN
1890,2016-04-02,Saturday,1,d_1891,NaN
1891,2016-04-03,Sunday,2,d_1892,NaN
1892,2016-04-04,Monday,3,d_1893,NaN


In [104]:
# Selecting the 30 last days
Tribeca = Tribeca[Tribeca.date >= "2016-03-26"]
# Grouping by items 
Tribeca = Tribeca[["id", "unit_sold"]].groupby(by="id").sum().sort_values(by='unit_sold',ascending=False)

In [108]:
# I will select the 10 h
items_selected = list(Tribeca.iloc[:50].index)

In [109]:
items_selected

['SUPERMARKET_3_090_NYC_3',
 'SUPERMARKET_3_586_NYC_3',
 'SUPERMARKET_3_120_NYC_3',
 'SUPERMARKET_3_252_NYC_3',
 'SUPERMARKET_3_282_NYC_3',
 'SUPERMARKET_3_362_NYC_3',
 'SUPERMARKET_3_288_NYC_3',
 'SUPERMARKET_3_681_NYC_3',
 'SUPERMARKET_3_587_NYC_3',
 'SUPERMARKET_3_499_NYC_3',
 'SUPERMARKET_3_714_NYC_3',
 'SUPERMARKET_3_555_NYC_3',
 'HOME_&_GARDEN_1_465_NYC_3',
 'SUPERMARKET_3_329_NYC_3',
 'SUPERMARKET_3_723_NYC_3',
 'HOME_&_GARDEN_1_459_NYC_3',
 'SUPERMARKET_3_202_NYC_3',
 'SUPERMARKET_3_233_NYC_3',
 'SUPERMARKET_3_739_NYC_3',
 'SUPERMARKET_3_318_NYC_3',
 'HOME_&_GARDEN_1_118_NYC_3',
 'HOME_&_GARDEN_1_334_NYC_3',
 'ACCESORIES_1_234_NYC_3',
 'HOME_&_GARDEN_1_418_NYC_3',
 'SUPERMARKET_3_694_NYC_3',
 'HOME_&_GARDEN_1_447_NYC_3',
 'HOME_&_GARDEN_1_110_NYC_3',
 'SUPERMARKET_3_281_NYC_3',
 'SUPERMARKET_3_458_NYC_3',
 'HOME_&_GARDEN_1_096_NYC_3',
 'HOME_&_GARDEN_1_535_NYC_3',
 'SUPERMARKET_3_389_NYC_3',
 'SUPERMARKET_2_360_NYC_3',
 'SUPERMARKET_3_263_NYC_3',
 'HOME_&_GARDEN_1_409_NYC_3',
 

In [14]:
def prophet_df(dataframe, items_selected):
    df_list = []
    for item in items_selected:       
        series = dataframe.loc[dataframe.id == item][['id', 'date', 'unit_sold']].set_index('id')
        ph_df = series.rename(columns={'date': 'ds', 'unit_sold':'y'})
        df_list.append(ph_df)
        
    return df_list     

In [111]:
df_list = prophet_df(dollar, items_selected)
df_list

[                                 ds    y
 id                                      
 SUPERMARKET_3_090_NYC_3  2011-01-29  108
 SUPERMARKET_3_090_NYC_3  2011-01-30  132
 SUPERMARKET_3_090_NYC_3  2011-01-31  102
 SUPERMARKET_3_090_NYC_3  2011-02-01  120
 SUPERMARKET_3_090_NYC_3  2011-02-02  106
 ...                             ...  ...
 SUPERMARKET_3_090_NYC_3  2016-04-20   88
 SUPERMARKET_3_090_NYC_3  2016-04-21   77
 SUPERMARKET_3_090_NYC_3  2016-04-22  141
 SUPERMARKET_3_090_NYC_3  2016-04-23  139
 SUPERMARKET_3_090_NYC_3  2016-04-24  130
 
 [1913 rows x 2 columns],
                                  ds   y
 id                                     
 SUPERMARKET_3_586_NYC_3  2011-01-29  56
 SUPERMARKET_3_586_NYC_3  2011-01-30  55
 SUPERMARKET_3_586_NYC_3  2011-01-31  45
 SUPERMARKET_3_586_NYC_3  2011-02-01  57
 SUPERMARKET_3_586_NYC_3  2011-02-02  54
 ...                             ...  ..
 SUPERMARKET_3_586_NYC_3  2016-04-20  70
 SUPERMARKET_3_586_NYC_3  2016-04-21  48
 SUPERMARKET_3_5

In [16]:
df_list[1].head()

,ds,y
id,,
ACCESORIES_1_008_NYC_3,2011-01-29,11
ACCESORIES_1_008_NYC_3,2011-01-30,12
ACCESORIES_1_008_NYC_3,2011-01-31,33
ACCESORIES_1_008_NYC_3,2011-02-01,13
ACCESORIES_1_008_NYC_3,2011-02-02,53


In [17]:
len(df_list)

567

## Modelisation

### Hyper parameter tuning

In [18]:
import itertools
import numpy as np
import pandas as pd
import random
from prophet.diagnostics import cross_validation, performance_metrics
param_grid= {
    'changepoint_prior_scale':[0.001, 0.01], 
    'seasonality_prior_scale':[0.01, 0.1]
}

all_params= [dict(zip(param_grid.keys(), values))
             for values in itertools.product(*param_grid.values())]
rmses= []

for params in all_params:
    m= Prophet(**params).fit(df_list[random.randint(0,566)])
    df_cv= cross_validation(m, horizon='30 days', parallel="processes")
    df_p= performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

tuning_results= pd.DataFrame(all_params)
tuning_results['rmses']= rmses

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 102 forecasts with cutoffs between 2012-01-31 00:00:00 and 2016-03-25 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x000001C203293D30>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 102 forecasts with cutoffs between 2012-01-31 00:00:00 and 2016-03-25 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x000001C203293760>
INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 102 forecasts with cutoffs between 2012-01-31 00:00:00 and 2016-03-25 00:00:00
INFO:prophet:Applying

AssertionError: 

In [19]:
best_params = all_params[np.argmin(rmses)]
print(best_params)

{'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 0.1}


### Predicting

In [20]:
import json
from prophet.serialize import model_to_json, model_from_json

In [112]:
def run_prophet(timeserie):
    model = Prophet(daily_seasonality=True,changepoint_prior_scale=0.001,seasonality_prior_scale=0.1)   
    model.add_country_holidays(country_name='US')
    model.fit(timeserie)
    forecast = model.make_future_dataframe(periods=28)
    forecast = model.predict(forecast)
    forecast["id"] =  timeserie.index[0]
    with open('serialized_model.json', 'w') as fout:
        json.dump(model_to_json(m), fout)  # Save model

    return forecast

In [113]:
start_time = time.time()
result = list(map(run_prophet,df_list))
print("--- %s seconds ---" % (time.time() - start_time))

--- 289.6464989185333 seconds ---


In [115]:
result

[             ds       trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
 0    2011-01-29  116.872705   71.507664  346.101087   116.872705   116.872705   
 1    2011-01-30  116.851832   57.413219  303.920865   116.851832   116.851832   
 2    2011-01-31  116.830959   34.366442  283.000585   116.830959   116.830959   
 3    2011-02-01  116.810087    6.361370  267.658390   116.810087   116.810087   
 4    2011-02-02  116.789214    9.771421  270.001021   116.789214   116.789214   
 ...         ...         ...         ...         ...          ...          ...   
 1936 2016-05-18   76.463096 -104.408867  159.345123    76.463096    76.463096   
 1937 2016-05-19   76.442223  -99.074131  163.844453    76.442223    76.442223   
 1938 2016-05-20   76.421351  -58.051328  198.998092    76.421351    76.421351   
 1939 2016-05-21   76.400478  -40.903380  216.967501    76.400478    76.400478   
 1940 2016-05-22   76.379605  -66.858754  187.182035    76.379605    76.379605   
 
       Christm

In [145]:
pred_df = pd.DataFrame()
for x in range(50):
    pred_df = pd.concat([result[x], pred_df], join='outer')
pred_df.to_csv("DS_market_data/Prediction_df.csv")

## Testing the model

In [119]:
# Loading the model 
with open('serialized_model.json', 'r') as fin:
    m = model_from_json(json.load(fin))  # Load model

In [126]:
# selecting a item from the hot items clusters randomlt
test_item =[Tribeca.index[-15]]

In [129]:
test_df = prophet_df(dollar,test_item)

In [130]:
test_df

[                                 ds  y
 id                                    
 SUPERMARKET_3_026_NYC_3  2011-01-29  0
 SUPERMARKET_3_026_NYC_3  2011-01-30  9
 SUPERMARKET_3_026_NYC_3  2011-01-31  7
 SUPERMARKET_3_026_NYC_3  2011-02-01  9
 SUPERMARKET_3_026_NYC_3  2011-02-02  7
 ...                             ... ..
 SUPERMARKET_3_026_NYC_3  2016-04-20  1
 SUPERMARKET_3_026_NYC_3  2016-04-21  2
 SUPERMARKET_3_026_NYC_3  2016-04-22  0
 SUPERMARKET_3_026_NYC_3  2016-04-23  1
 SUPERMARKET_3_026_NYC_3  2016-04-24  2
 
 [1913 rows x 2 columns]]

In [131]:
test_forecast = m.predict(test_df[0])

In [132]:
plot_plotly(m, test_forecast)

In [133]:
df_cv = cross_validation(m, initial='730 days', period='180 days', horizon = '365 days')

INFO:prophet:Making 5 forecasts with cutoffs between 2013-05-05 00:00:00 and 2015-04-25 00:00:00
100%|██████████| 5/5 [00:23<00:00,  4.68s/it]


In [134]:
df_p = performance_metrics(df_cv)
df_p

INFO:prophet:Skipping MAPE because y close to 0


,horizon,mse,rmse,mae,mdape,smape,coverage
0,37 days,10.245791,3.200905,2.658451,0.406619,0.567087,0.673626
1,38 days,10.189782,3.192144,2.654287,0.404000,0.567568,0.671429
2,39 days,10.250552,3.201648,2.665167,0.410912,0.575652,0.665934
3,40 days,10.361780,3.218972,2.680030,0.418759,0.578214,0.665934
4,41 days,10.251611,3.201814,2.661165,0.410912,0.571603,0.673626
...,...,...,...,...,...,...,...
324,361 days,18.290298,4.276716,3.434938,0.511599,0.671996,0.550549
325,362 days,18.373587,4.286442,3.447662,0.508607,0.668656,0.545055
326,363 days,18.617749,4.314829,3.481714,0.508607,0.677144,0.539560
327,364 days,19.235239,4.385800,3.537689,0.520062,0.691254,0.530769
